In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import os
dftr = pd.read_csv("../input/feedback-prize-effectiveness/train.csv")
print(dftr.shape,dftr.discourse_id.nunique())
dfte = pd.read_csv("../input/feedback-prize-effectiveness/test.csv")
print(dfte.shape,dfte.discourse_id.nunique())
dfs = pd.read_csv("../input/feedback-prize-effectiveness/sample_submission.csv")

dftr["text"] = dftr["essay_id"].apply(lambda x: open(f'../input/feedback-prize-effectiveness/train/{x}.txt').read())
dfte["text"] = dfte["essay_id"].apply(lambda x: open(f'../input/feedback-prize-effectiveness/test/{x}.txt').read())
FOLDS = 5
from sklearn.model_selection import StratifiedKFold
target_map = {"Adequate":1,"Effective":2,"Ineffective":0}
dftr["target"] = dftr["discourse_effectiveness"].map(target_map)
dftr = dftr.reset_index(drop=True)
# print(dftr["target"].value_counts())
skf = StratifiedKFold(n_splits=FOLDS,shuffle=True,random_state=FOLDS)
for i,(train_index, test_index) in enumerate(skf.split(dftr, dftr["target"])):
    dftr.loc[test_index,"fold"] = i
print(dftr.fold.value_counts())    


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from scipy import sparse
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
import numpy as np
preds = []
for fold in range(FOLDS):
    dftr_ = dftr[dftr["fold"]!=fold]
    dfev_ = dftr[dftr["fold"]==fold]
    tf = TfidfVectorizer(ngram_range=(1,2))
    tr_text_feats = tf.fit_transform(dftr_["discourse_text"])
    ev_text_feats = tf.transform(dfev_["discourse_text"])
    te_text_feats = tf.transform(dfte["discourse_text"])
    tf = TfidfVectorizer(ngram_range=(1,2))
    tr_text_feats2 = tf.fit_transform(dftr_["text"])
    ev_text_feats2 = tf.transform(dfev_["text"])
    te_text_feats2 = tf.transform(dfte["text"])
    ohe = OneHotEncoder()
    tr_feats1 = sparse.csr_matrix(ohe.fit_transform(dftr_["discourse_type"].values.reshape(-1,1)))
    ev_feats1 = sparse.csr_matrix(ohe.transform(dfev_["discourse_type"].values.reshape(-1,1)))
    te_feats1 = sparse.csr_matrix(ohe.transform(dfte["discourse_type"].values.reshape(-1,1)))
    tr_feats = sparse.hstack((tr_feats1,tr_text_feats,tr_text_feats2))
    ev_feats = sparse.hstack((ev_feats1,ev_text_feats,ev_text_feats2))
    te_feats = sparse.hstack((te_feats1,te_text_feats,te_text_feats2))
    clf = LogisticRegression(max_iter=500,penalty="l2",C=1.0131816333513533)
    clf.fit(tr_feats, dftr_["target"].values)
    ev_preds = clf.predict_proba(ev_feats)
    ev_loss = log_loss(dfev_["target"].values,ev_preds)
    print("Fold : {} EV score: {}".format(fold,ev_loss))
    preds.append(clf.predict_proba(te_feats))
    # break

In [ ]:
all_preds = np.array(preds).mean(0)
print(all_preds.shape)
dfs.loc[:,"Ineffective"] = all_preds[:,0]
dfs.loc[:,"Adequate"] = all_preds[:,1]
dfs.loc[:,"Effective"] = all_preds[:,2]
dfs.head()

In [ ]:
dfs.to_csv('submission.csv',index=None)